### Step 1:

In [4]:
#Import libraries
import pandas as pd
import numpy as np
import os

In [5]:
#import the orders_products_combined
ords_prods_merge = pd.read_pickle(...\02 Data\Prepared Data\orders_products_combined.pkl')
path = ...
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_combined.pkl'))

### Step 2:

In [9]:
# Calculate the mean of 'order_number' grouped by 'department_id'
dept_order_mean = ords_prods_merge.groupby('department_id')['order_number'].mean()
print(dept_order_mean)

department_id
1     16.559358
2     18.413176
3     18.279600
4     18.915890
5     16.497751
6     17.609390
7     18.303975
8     16.383301
9     17.022963
10    21.227447
11    17.311768
12    16.953613
13    17.727470
14    17.898097
15    17.290123
16    18.757796
17    16.808683
18    20.336389
19    18.258620
20    17.578959
21    23.644477
Name: order_number, dtype: float64


### Step 3:

##### Analysis of Results:
- The results for the full DataFrame provide a more comprehensive view of the average `order_number` across departments compared to the subset, which could have biased results due to limited data.
- Departments with the highest and lowest averages indicate differences in customer behavior across product categories.


### Step 4:

In [24]:
#Use transform() and loc() to create a loyalty flag for customers.
# Calculate the maximum orders per user
ords_prods_merge['max_order'] = ords_prods_merge.groupby('user_id')['order_number'].transform(np.max)

...\ipykernel_7712\1321159053.py:3: FutureWarning: The provided callable <function max at 0x00000216D07A4360> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby('user_id')['order_number'].transform(np.max)


In [25]:
# Create loyalty flag
ords_prods_merge['loyalty_flag'] = 'New customer'
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] > 10) & (ords_prods_merge['max_order'] <= 40), 'loyalty_flag'] = 'Regular customer'
print(ords_prods_merge['loyalty_flag'].value_counts(dropna = False))

loyalty_flag
Regular customer    15081691
Loyal customer      10095381
New customer         5151691
Name: count, dtype: int64


### Step 5:

In [26]:
# Group by loyalty_flag and calculate statistics for prices
loyalty_price_stats = ords_prods_merge.groupby('loyalty_flag')['prices'].describe()
print(loyalty_price_stats)

                       count       mean         std  min  25%  50%   75%  \
loyalty_flag                                                               
Loyal customer    10095381.0  10.402162  329.036155  1.0  4.2  7.4  11.2   
New customer       5151691.0  13.396333  604.763039  1.0  4.2  7.4  11.3   
Regular customer  15081691.0  12.546842  545.957947  1.0  4.2  7.4  11.3   

                      max  
loyalty_flag               
Loyal customer    99999.0  
New customer      99999.0  
Regular customer  99999.0  


### Step 6:

In [29]:
#Create a spending flag
# Calculate the mean price per user
ords_prods_merge['avg_price'] = ords_prods_merge.groupby('user_id')['prices'].transform('mean')

In [31]:
# Create spending flag
ords_prods_merge['spending_flag'] = 'Low spender'
ords_prods_merge.loc[ords_prods_merge['avg_price'] >= 10, 'spending_flag'] = 'High spender'
print(ords_prods_merge['spending_flag'].value_counts())

spending_flag
Low spender     29730505
High spender      598258
Name: count, dtype: int64


### Step 7:

In [34]:
#Create an Order frequency flag
# Calculate the median days_since_prior_order per user
ords_prods_merge['median_days_since_prior_order'] = ords_prods_merge.groupby('user_id')['days_since_prior_order'].transform('median')

# Create order frequency flag
ords_prods_merge['order_frequency_flag'] = 'Frequent customer'
ords_prods_merge.loc[ords_prods_merge['median_days_since_prior_order'] > 20, 'order_frequency_flag'] = 'Non-frequent customer'
ords_prods_merge.loc[(ords_prods_merge['median_days_since_prior_order'] > 10) & (ords_prods_merge['median_days_since_prior_order'] <= 20), 'order_frequency_flag'] = 'Regular customer'
print(ords_prods_merge['order_frequency_flag'].value_counts())

order_frequency_flag
Frequent customer        20675290
Regular customer          6594542
Non-frequent customer     3058931
Name: count, dtype: int64


### Step 8:

In [39]:
# Check for missing values
missing_values = ords_prods_merge.isnull().sum()
print("Missing values per column:")
print(missing_values)

Missing values per column:
order_id                         0
user_id                          0
order_number                     0
orders_day_of_week               0
order_hour_of_day                0
days_since_prior_order           0
product_id                       0
add_to_cart_order                0
reordered                        0
product_name                     0
aisle_id                         0
department_id                    0
prices                           0
busiest_day                      0
day_summary                      0
busiest_period_of_day            0
max_order                        0
loyalty_flag                     0
avg_price                        0
spending_flag                    0
median_days_since_prior_order    0
order_frequency_flag             0
dtype: int64


In [38]:
print(ords_prods_merge.dtypes)

order_id                            int64
user_id                             int64
order_number                        int64
orders_day_of_week                  int64
order_hour_of_day                   int64
days_since_prior_order            float64
product_id                          int64
add_to_cart_order                   int64
reordered                           int64
product_name                       object
aisle_id                            int64
department_id                       int64
prices                            float64
busiest_day                      category
day_summary                      category
busiest_period_of_day            category
max_order                           int64
loyalty_flag                       object
avg_price                         float64
spending_flag                      object
median_days_since_prior_order     float64
order_frequency_flag               object
dtype: object


In [40]:
# Convert columns with categorical data to 'category' type for efficiency
ords_prods_merge['product_name'] = ords_prods_merge['product_name'].astype('category')
ords_prods_merge['busiest_day'] = ords_prods_merge['busiest_day'].astype('category')
ords_prods_merge['day_summary'] = ords_prods_merge['day_summary'].astype('category')
ords_prods_merge['busiest_period_of_day'] = ords_prods_merge['busiest_period_of_day'].astype('category')
ords_prods_merge['loyalty_flag'] = ords_prods_merge['loyalty_flag'].astype('category')
ords_prods_merge['spending_flag'] = ords_prods_merge['spending_flag'].astype('category')
ords_prods_merge['order_frequency_flag'] = ords_prods_merge['order_frequency_flag'].astype('category')
# Ensure all relevant columns with numeric data are appropriately typed
ords_prods_merge['order_hour_of_day'] = ords_prods_merge['order_hour_of_day'].astype(int)
ords_prods_merge['orders_day_of_week'] = ords_prods_merge['orders_day_of_week'].astype(int)

In [41]:
# Drop intermediate columns that are no longer needed
ords_prods_merge = ords_prods_merge.drop(columns=['max_order', 'avg_price'], errors='ignore')  # Ignore errors if columns don't exist

In [43]:
# Export the cleaned DataFrame to a pickle file for further use
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_combined.pkl'))